In [38]:
import time
import requests
import pandas as pd

In [6]:
top_songs = pd.read_csv('../data/interim/top_songs_by_year.csv')
all_songs = pd.read_csv('../data/interim/all_songs.csv')

print(top_songs.head())
print(all_songs.head())

      playlist  track_id  year
0  Replay 2015      1424  2015
1  Replay 2015      1412  2015
2  Replay 2015      3587  2015
3  Replay 2015      1506  2015
4  Replay 2015      2247  2015
   Track ID                 Title               Artist  \
0      5253            Wanna Know            Mya Lanae   
1      5251               Anxiety  Megan Thee Stallion   
2      5171           Villain Era             Peachkka   
3      5153            Innerstood         Shariya Wise   
4      5157  Sweet Like Candyland         Shariya Wise   

                                    Composer                 Album  \
0  Ethan Smith Jr., Abel Herrera & Mya Smith   Wanna Know - Single   
1                   Kevin Price & Megan Pete            Traumazine   
2        Nikkole Maresa Baker & Kerby Joseph  Villain Era - Single   
3                     Ashlee Shariya Lombard   Baddie Affirmations   
4                     Ashlee Shariya Lombard   Baddie Affirmations   

         Genre         Release Date         

In [10]:
all_songs = all_songs.rename(columns={
    'Track ID': 'track_id',
    'Release Date': 'release_date',
    'Total Time (ms)': 'total_time_ms',
    'Date Modified': 'date_modified',
    'Date Added': 'date_added'})

top_songs_detail = top_songs.merge(all_songs, on='track_id', how='left')
top_songs_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026 entries, 0 to 1025
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   playlist             1026 non-null   object 
 1   track_id             1026 non-null   int64  
 2   year                 1026 non-null   int64  
 3   Title                1026 non-null   object 
 4   Artist               656 non-null    object 
 5   Composer             597 non-null    object 
 6   Album                656 non-null    object 
 7   Genre                656 non-null    object 
 8   release_date         332 non-null    object 
 9   Kind                 636 non-null    object 
 10  total_time_ms        656 non-null    float64
 11  date_modified        656 non-null    object 
 12  date_added           421 non-null    object 
 13  Release Date (Raw)   332 non-null    object 
 14  Date Modified (Raw)  656 non-null    object 
 15  Date Added (Raw)     421 non-null    o

In [14]:
# Get access token
def get_spotify_token():
    url = "https://accounts.spotify.com/api/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {"grant_type": "client_credentials"}
    response = requests.post(url, headers=headers, data=data, auth=(CLIENT_ID, CLIENT_SECRET))
    return response.json().get("access_token")

In [40]:
# Search Spotify for a track
def search_spotify(title, artist, token):
    query = f"track:{title}"
    if pd.notna(artist):  # Only add artist if it's not NaN
        query += f" artist:{artist}"
    
    url = f"https://api.spotify.com/v1/search?q={query}&type=track&limit=1"
    headers = {"Authorization": f"Bearer {token}"}

    for attempt in range(5):  # Retry up to 5 times
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:  # Rate limit hit
            retry_after = int(response.headers.get("Retry-After", 5))  # Default 5 sec
            print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
            time.sleep(retry_after)
        else:
            print(f"Error {response.status_code}: {response.text}")
            time.sleep(2)  # Wait before retrying
    
    return None

In [18]:
# Example usage
token = get_spotify_token()
song_info = search_spotify("Shape of You", "Ed Sheeran", token)
print((song_info))

{'tracks': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=1&query=track%3AShape%20of%20You%20artist%3AEd%20Sheeran&type=track', 'limit': 1, 'next': 'https://api.spotify.com/v1/search?offset=1&limit=1&query=track%3AShape%20of%20You%20artist%3AEd%20Sheeran&type=track', 'offset': 0, 'previous': None, 'total': 100, 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6eUKZXaKkcviH0Ku9w2n3V'}, 'href': 'https://api.spotify.com/v1/artists/6eUKZXaKkcviH0Ku9w2n3V', 'id': '6eUKZXaKkcviH0Ku9w2n3V', 'name': 'Ed Sheeran', 'type': 'artist', 'uri': 'spotify:artist:6eUKZXaKkcviH0Ku9w2n3V'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY',

In [50]:
song_info['tracks']['items'][0].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'is_playable', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [56]:
if song_info['tracks'] :
    print('hi')

hi


In [46]:
song_info['tracks']['items'][0]['artists'][0]['name']

'Ed Sheeran'

In [58]:
{'spotify_id': song_info['tracks']['items'][0]['id'],
            'artist': song_info['tracks']['items'][0]['artists'][0]['name'],
            'title': song_info['tracks']['items'][0]['name'],
            'popularity': song_info['tracks']['items'][0]['popularity'],
        }

{'spotify_id': '7qiZfU4dY1lWllzX7mPBI3',
 'artist': 'Ed Sheeran',
 'title': 'Shape of You',
 'popularity': 89}

In [62]:
# Remove duplicates to ensure unique track_id queries
unique_tracks = top_songs_detail[['track_id', 'Title', 'Artist']].drop_duplicates()
unique_tracks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 694 entries, 0 to 1025
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   track_id  694 non-null    int64 
 1   Title     694 non-null    object
 2   Artist    396 non-null    object
dtypes: int64(1), object(2)
memory usage: 21.7+ KB


In [80]:

# Store results
results = []
missing = []
token = get_spotify_token()
# Loop through unique tracks and query Spotify API
for i, row in unique_tracks.iterrows():
    track_id = row["track_id"]
    title = row["Title"]
    artist = row["Artist"]

    print(f"Searching for: {title} by {artist}")
    song_info = search_spotify(title, artist, token)

    # Check if 'tracks' exists and has items before accessing
    if song_info.get('tracks') and song_info['tracks'].get('items'):
        first_item = song_info['tracks']['items'][0]
        results.append({
            'apple_id': track_id,
            'spotify_id': first_item['id'],
            'artist': first_item['artists'][0]['name'],
            'title': first_item['name'],
            'popularity': first_item['popularity'],
        })
    else:
        print(f"No results found for {title} by {artist}")
        missing.append(track_id)

    # Introduce delay every 50 requests to avoid rate limits
    if (i + 1) % 50 == 0:
        print("Pausing to avoid rate limits...")
        time.sleep(10)  # Pause for 10 seconds

    # Small delay between requests to reduce the risk of hitting rate limits
    time.sleep(0.5)

Searching for: Highway Don't Care (feat. Taylor Swift & Keith Urban) by nan
Searching for: I Knew You Were Trouble by nan
Searching for: Girl Crush by Little Big Town
Searching for: Sangria by nan
Searching for: Forever & Always by nan
Searching for: Love Story by nan
Searching for: Yoga by Janelle Monáe & Jidenna
Searching for: Sure Be Cool If You Did by Blake Shelton
Searching for: Honey Bee by Blake Shelton
Searching for: Our Song by nan
Searching for: Hillbilly Bone (feat. Trace Adkins) by nan
Searching for: Mine Would Be You by nan
Searching for: Back to December by nan
Searching for: Need You Now by nan
Searching for: We Are Never Ever Getting Back Together by Taylor Swift
Searching for: Doin' What She Likes by nan
Searching for: Photograph by Ed Sheeran
Searching for: The A Team by Ed Sheeran
Searching for: Play It Again by nan
Searching for: Don't by Ed Sheeran
Searching for: Bless the Broken Road by Rascal Flatts
Searching for: Just A Kiss by nan
Searching for: Begin Again by 

In [82]:
len(missing)

36

In [84]:
len(results)

658

In [94]:
missing_df= pd.DataFrame(missing)
results_df= pd.DataFrame(results)

In [98]:
missing_df.to_csv('../data/interim/spotify_search_missing.csv')
results_df.to_csv('../data/interim/spotify_search_results.csv')